In [35]:
import pickle
import pandas as pd
#deserialize data from a file
def load_data(file_name,language):
    path="../data/"+language+"/"+file_name
    try: 
        file = open(path, 'rb') 
        data = pickle.load(file) 
        return data
    except: 
        print("Error in reading data")

#caricamento delle probabilità apprese
emission_prob=load_data("emission_prob","en")
transition_prob=load_data("transition_prob","en")
unkown_prob=load_data("unknown_prob","en")

Viterbi

In [66]:
import pandas as pd
import numpy as np

# Funzione per l'algoritmo di Viterbi: prende in input le probabilità di emissione e di transizione sottoforma di dataframe
# e restituisce le coppie parola-NER_TAG assegnate
def Viterbi(emission_dataframe, transition_dataframe):
    epsilon = 1e-10  # Valore molto piccolo
    # Lista delle parole nel testo
    testo_array = emission_dataframe.columns.tolist()
    # Numero di stati (tag)
    K = transition_dataframe.shape[0]
    
    # Lunghezza del testo
    T = len(testo_array)
    
    # Inizializzazione
    viterbi = np.zeros((K, T))
    backpointer = np.zeros((K, T), dtype=int)
    
    # Inizializzazione del primo passo
    Pi = 1  # Start equiprobabile 
    viterbi[:, 0] = np.log(emission_dataframe.iloc[:, 0] + epsilon) + np.log(Pi)
    backpointer[:, 0] = 0 
    
    # Iterazione
    for t in range(1, T):
        for k in range(K):
            # Calcolo della probabilità Viterbi per ogni stato
            max_val = float('-inf')
            max_idx = 0
            for j in range(K):
                val = viterbi[j, t - 1] + np.log(transition_dataframe.iloc[j, k] + epsilon) + np.log(emission_dataframe.iloc[k, t] + epsilon)
                if val > max_val:
                    max_val = val
                    max_idx = j
            viterbi[k, t] = max_val
            backpointer[k, t] = max_idx
    
    # Terminazione: Trova il massimo della colonna finale
    max_final_val = np.max(viterbi[:, T - 1])
    max_final_idx = np.argmax(viterbi[:, T - 1])
    
    # Costruzione del percorso Viterbi partendo dall'ultimo passo
    viterbi_path = [max_final_idx]
    for i in reversed(range(1, T)):
        viterbi_path.insert(0, backpointer[viterbi_path[0], i])
    
    # Costruzione delle coppie parola-tag assegnate
    word_tag_pairs = [(testo_array[i], transition_dataframe.index[j]) for i, j in enumerate(viterbi_path)]
    
    # Stampa del percorso Viterbi e delle coppie parola-tag assegnate
    print("Percorso Viterbi:", viterbi_path)
    print("Coppie parola-tag assegnate:", word_tag_pairs)
    
    return word_tag_pairs

# Esempio di utilizzo

# viterbi_path = Viterbi(emission_prob, transition_prob)


Creazione del sub-dataset di probabilità di emissione per le parole di una frase.

Applicazione di diverse tecniche di smoothing per gestire le parole sconosciute:

1 - Sempre O: P(unk|O) = 1

2 - Sempre O o MISC: P(unk|O)=P(unk|B-MISC)=0.5

3 - Uniforme: P(unk|tag) = 1/#(NER_TAGs)

4 - Statistica TAG sul val set: parole che compaiono 1 sola volta  -> unknown_prob calcolata nel file learning 


In [69]:
#prende in input una frase, le probabilità di emisione e transizione apprese 
#restituisce le coppie parola-NER_TAG assegnate utilizzano l'algoritmo di Viterbi e applicando la tecnica di smoothing specificata
def viterbi_tagger(sentence, emission_prob, transition_prob, smoothing_type=1):
    #inizializzazione
    tags=transition_prob.keys()
    words = sentence.split()
    transition_df = pd.DataFrame.from_dict(transition_prob)
    emission_sentence_df = pd.DataFrame(columns=words,index=tags)
    
    #iterazione per ogni parola delle frase aggiorna il dataframe delle emissioni
    for word in words:
        if word in emission_prob:
            emission_sentence_df[word] = emission_prob[word].values()
        else: #applicazione dello smoothing
            if (smoothing_type==1): emission_sentence_df[word] =  {tag: 1 if tag == "O" else 0 for tag in tags}
            elif (smoothing_type==2): emission_sentence_df[word] =  {tag: 0.5 if tag == "B-MISC" or tag == "O" else 0 for tag in tags}
            elif (smoothing_type==3): emission_sentence_df[word] =  {tag: 1/len(tags) for tag in tags}
            elif (smoothing_type==4): emission_sentence_df[word] =  unkown_prob
   
    return Viterbi(emission_sentence_df, transition_df)

# Esempio di utilizzo
sentence = "The PAROLASCONOSCIUTA tour would cover the United States and the UK and Ireland throughout 2019 ."
viterbi_tagger(sentence,emission_prob,transition_prob,4)

Percorso Viterbi: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
Coppie parola-tag assegnate: [('The', 'O'), ('PAROLASCONOSCIUTA', 'O'), ('tour', 'O'), ('would', 'O'), ('cover', 'O'), ('the', 'O'), ('United', 'B-LOC'), ('States', 'O'), ('and', 'O'), ('the', 'O'), ('UK', 'B-LOC'), ('and', 'O'), ('Ireland', 'B-LOC'), ('throughout', 'O'), ('2019', 'O'), ('.', 'O')]


[('The', 'O'),
 ('PAROLASCONOSCIUTA', 'O'),
 ('tour', 'O'),
 ('would', 'O'),
 ('cover', 'O'),
 ('the', 'O'),
 ('United', 'B-LOC'),
 ('States', 'O'),
 ('and', 'O'),
 ('the', 'O'),
 ('UK', 'B-LOC'),
 ('and', 'O'),
 ('Ireland', 'B-LOC'),
 ('throughout', 'O'),
 ('2019', 'O'),
 ('.', 'O')]